# 1. Question

In [ ]:
import requests
rss_url = "http://fs.jtbc.joins.com/RSS/economy.xml"
jtbc_economy = requests.get(rss_url)

In [ ]:
from bs4 import BeautifulSoup
economy_news_list = BeautifulSoup(jtbc_economy.content, "xml")
link_list = economy_news_list.select("item > link")

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
news =[]
for link in link_list:
    news_url = link.text
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    news_content = news_soup.select_one("#articlebody > .article_content")
    news.append(list(filter(lambda word: len(word)>1, mecab.nouns(news_content.text))))
    
news

# 2. Question

In [ ]:
word_Korean = ["추석", "연휴", "민족", "대이동", "시작", "늘어", "교통량", "교통사고", "특히", "자동차", \
               "고장", "상당수", "차지", "나타", "것", "기자"]

In [ ]:
stopwords = ["가다", "늘어", "나타", "것", "기자"] 

In [ ]:
results = [word for word in word_Korean if word not in stopwords]
results

# 3. Question

In [ ]:
pip install pymongo

In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver as wd
import re
import pandas as pd
import time

title_list = list()
point_list = list()
regdate_list = list()
content_list = list()


for i in range(1, 4):
    #for linux
    driver = wd.Chrome(executable_path = './chromedriver')
    #for Windows
    #driver = wd.Chrome(executable_path = 'chromedriver.exe')
    url = 'http://www.yes24.com/Product/communityModules/GoodsReviewList/40936880?Sort=1&PageNumber='
    url += str(i) + '&Type=ALL&_=156817'
    driver.get(url)

    # #암묵적 대기
    driver.implicitly_wait(2)
    #제목, 평점, 작성날짜, 리뷰내용
    #review_Tit,review_rating, txt_date, review_cont
    time.sleep(2)
    review_info_group = driver.find_elements_by_css_selector('.reviewInfoGrp')
    for review_info in review_info_group:
        title = review_info.find_element_by_css_selector('.review_Tit').text
        review_rating = review_info.find_element_by_css_selector('.review_rating').text  #내용 평점5점   편집/디자인 평점5점
        txt_date = review_info.find_element_by_css_selector('.txt_date').text
        review_cont = review_info.find_element_by_css_selector('.review_cont').text
        title_list.append(title)
        point = re.sub(r'평점', '', review_rating.split(' ')[1])
        point_list.append(re.sub(r'점', '', point))
        regdate_list.append(txt_date)
        content_list.append(review_cont)
    driver.close()

df = pd.DataFrame(title_list, columns = ['title'])
df['point'] = point_list
df['regdate'] = regdate_list
df['review'] = content_list

import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.test

try:
    for i in range(0, len(title_list)):
        db.insight.insert({
            'title' : df.loc[i, 'title'],
            'point' : df.loc[i, 'point'],
            'regdate' : df.loc[i, 'regdate'],
            'review' : df.loc[i, 'review']
        })
    print("Success.")
except Exception as e: 
    print('Exception 발생 = ', e)


In [ ]:
review_list = list()
reviews = db.insight.find({}, {'review':1, '_id':0})
for review in reviews:
    review_list.append(review)
    
print(review_list)

In [ ]:
text = ''
reviews = db.insight.find({}, {'review':1, '_id':0})
for review in reviews:
    text += review['review']

text
from konlpy.tag import Mecab
mecab = Mecab()
nouns = mecab.nouns(text)
nouns_list = [noun for noun in nouns if len(noun) > 2]
text = ' '.join(nouns_list)

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
#for Linux
cloud = WordCloud(background_color = 'white')
#for Windows
#cloud = WordCloud(background_color = 'white', \
#                  font_path='C:/Windows/Fonts/magungitic.ttf')

In [ ]:
cloud.generate(text)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(cloud, interpolation = 'bilinear')
plt.axis('off')